In [7]:
# -*- coding: utf-8 -*-
import cv2
import os
import sys
import random
import json
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import ipdb

In [8]:
def get_infos(source_lst, num_samples, size):
    recs = open(source_lst,"r").readlines()
    label_lst = []
    image_lst = []
    recs = random.sample(recs, num_samples)
    for line in recs:
        if "points" in line:
            json_str = json.loads(line.split("\t")[-2])
            path = line.split("\t")[0]
            original_im = cv2.imread(path)
            h, w, _ = original_im.shape
            xs, ys = zip(*json_str["points"])
            xmin, ymin = min(xs), min(ys)
            xmax, ymax = max(xs), max(ys)
            xmin *= w
            xmax *= w
            ymin *= h
            ymax *= h
            img = original_im[int(ymin):int(ymax), int(xmin):int(xmax)]
            label = int(line.split("\t")[1])
        else:
            path = line.split(" ")[0]
            img = cv2.imread(path)
            
            label = int(line.split(" ")[1])
        if label in [11, 13]:continue
        age_mapping = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 12:6, 6:7, 7:8, 8:9, 9:10, 10:11}
        label = age_mapping[label]
        img = cv2.resize(img,(size[1], size[0]), interpolation=cv2.INTER_CUBIC)
        label_lst.append(label)
        image_lst.append(img)
    return image_lst, label_lst
    

In [9]:
h_gap = 50
v_gap = 50
text_gap = 45
rows = 8
cols = 7
num_samples = rows*cols
img_size = (256, 128)

source_lst = "/world/data-c26/liliang/person_attribute/v4_20190514/lst/age/source/train_keypoints_v4.json"
batch_size = num_samples
page = np.ones(((img_size[0]+h_gap)*rows, (img_size[1]+v_gap)*cols, 3), dtype=np.uint8) * 255
normalize_mean = np.reshape(np.array([0.485, 0.456, 0.406]),
                                         [3, 1, 1])
normalize_variance = np.reshape(np.array([0.229, 0.224, 0.225]),
                                             [3, 1, 1])
image_lst, label_lst = get_infos(source_lst, num_samples, img_size)
# for img in imgs:
#     img=np.array(img).astype(np.float32)
#     img = img*normalize_variance+normalize_mean
#     img*=255.0
#     img = np.transpose(img, [1,2,0]).astype(np.float32)
#     np_imgs.append(img)
k=0
age_scpoe_mapping = {0:"0-1", 1:"2-5", 2:"6-10", 3:"11-15", 4:"16-20", 
                    5:"21-25", 6:"25-30", 7:"31-40", 8:"41-50", 
                     9:"51-60", 10:"61-80", 11:"80+", }
for row in range(1,rows+1):
    for col in range(1,cols+1):
        label = label_lst[col*(row-1)+col-1]
        cv2.putText(page, str(label), \
                    ((col-1)*(img_size[1]+v_gap), (row-1)*(h_gap+img_size[0])+50), \
                    cv2.FONT_HERSHEY_PLAIN, 1.5, (0,0,0), 2)
        cv2.putText(page, age_scpoe_mapping[label], \
            ((col-1)*(img_size[1]+v_gap)+60, (row-1)*(h_gap+img_size[0])+50), \
            cv2.FONT_HERSHEY_PLAIN, 1.5, (0,0,0), 2)
        page[row*h_gap+(row-1)*img_size[0]:row*(h_gap+img_size[0])\
            , (col-1)*(v_gap+img_size[1]):col*img_size[1]+(col-1)*v_gap] \
            = image_lst[k]
        k+=1

plt.figure(figsize = (30,30))
plt.axis('off')
plt.imshow(cv2.cvtColor(page, cv2.COLOR_BGR2RGB))

IndexError: list index out of range